# COVID Epidemic Data Analysis

**Author**: Ravinder Mallarapu

**Date**: 20200422

**Description**: This notebook presents JHU COVID data preparation, SIER epidemic forecast modeling, descriptive analytics, and data visualization.



## Process JHU Daily COVID Report

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def get_daily_covid_stats():
    """return daily COVID dataframe"""
    today = datetime.today()
    yesterday = today + timedelta(days=-1)
    daybeforeyesterday = today + timedelta(days=-2)
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/' \
          'csse_covid_19_data/csse_covid_19_daily_reports/'
    
    # accomodate variation in different time zones where today is still yesterday
    try:
        endpoint = url + f'{today.month:02}-{today.day:02}-{today.year}.csv'
        df = pd.read_csv(endpoint)
    except:
        try:
            endpoint = url + f'{yesterday.month:02}-{yesterday.day:02}-{yesterday.year}.csv'
            df = pd.read_csv(endpoint)
        except:
            endpoint = url + f'{daybeforeyesterday.month:02}-{daybeforeyesterday.day:02}-{daybeforeyesterday.year}.csv'
            df = pd.read_csv(endpoint)

    return df

In [3]:
daily = get_daily_covid_stats().groupby("Country_Region").sum()

## Top-10: Worst Infected Countries

In [4]:
daily.sort_values(by='Confirmed', ascending=False).head(10)

,FIPS,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
Country_Region,,,,,,,,,
US,105902623.0,121693.134039,-293352.619164,45050910,725835,0.0,0.0,4.576023e+07,7805.446659
India,0.0,831.177882,2945.020567,34094373,452454,0.0,0.0,1.480622e+05,45.762292
Brazil,0.0,-342.077100,-1308.973300,21651910,603465,0.0,0.0,3.170969e+05,67.130793
United Kingdom,0.0,422.469698,-509.497466,8537650,139042,0.0,0.0,1.197337e+05,15.439266
Russia,0.0,4527.343882,5156.031399,7903963,220323,0.0,0.0,4.479371e+05,235.480077
Turkey,0.0,38.963700,35.243300,7683487,67837,0.0,0.0,9.110235e+03,0.882893
France,0.0,77.169695,-416.763414,7190716,118232,0.0,0.0,1.133501e+05,14.071201
Iran,0.0,32.427908,53.688046,5796659,124256,0.0,0.0,6.901363e+03,2.143580
Argentina,0.0,-38.416100,-63.616700,5273463,115704,0.0,0.0,1.166804e+04,2.194080


## Epidemic Modeling Using SEIR

#### SEIR helper functions

In [5]:
import numpy as np
from scipy.integrate import solve_ivp

class DiffSEIR:
    def seir(self, t, y):
        N = y[:3].sum()
        return np.array([
            -self.beta * y[0] * y[2] / N,
            self.beta * y[0] * y[2] / N - self.sigma * y[1],
            self.sigma * y[1] - self.gamma * y[2]  - self.mu * y[2],
            self.gamma * y[2],
            self.mu * y[2]
        ])

    def __call__(self, beta, gamma, sigma, mu, S, E, I, R, D, days):
        self.beta, self.gamma, self.sigma, self.mu = beta, gamma, sigma, mu
        y = np.array([S, E, I, R, D])

        sol = solve_ivp(self.seir, [0, days], y, t_eval = range(0, days + 1))

        return sol.y.transpose()

In [6]:
def run_seir(*args, **kwargs):
    seir = DiffSEIR()
    return pd.DataFrame(data=seir(*args, **kwargs), 
                        columns=['Susceptible', 'Exposed', 'Infected', 'Recovered', 'Dead'])

#### Modeling True Infected Cases for the US Using SEIR

In [7]:
population = 328000000  # US population
country_data = daily.loc["US"]
initial_infected = country_data["Confirmed"]
initial_recovered = country_data["Recovered"]
initial_deaths = country_data["Deaths"]

In [8]:
# reference: https://science.sciencemag.org/content/early/2020/03/13/science.abb3221
mu = 0.00645
days = 300
symptomatic_contact_rate = 5 # assuming a person will come in contact with 5 COVID patients a day
initial_exposed = initial_infected * 0.14
initial_susceptible = population - initial_exposed - initial_infected - initial_deaths
gamma = 17.8 # number of days to recover
sigma = 17.8 # number of days to incubate
trans = 0.018 # 
beta = symptomatic_contact_rate * trans
gamma = 1 / gamma
sigma = 1 / sigma

In [9]:
# Run SEIR model on input parameters
state = run_seir(
    beta, gamma, sigma, mu,
    initial_susceptible, initial_exposed,
    initial_infected, initial_recovered, initial_deaths, days + 1
)
state.index.name = 'Days'
state = state.reset_index().melt('Days')
state.rename(columns={"variable": "Status", "value": "Forecast"}, inplace=True)

In [10]:
state

,Days,Status,Forecast
0,0,Susceptible,2.759161e+08
1,1,Susceptible,2.725935e+08
2,2,Susceptible,2.694451e+08
3,3,Susceptible,2.664499e+08
4,4,Susceptible,2.635897e+08
...,...,...,...
1505,297,Dead,3.436240e+07
1506,298,Dead,3.436517e+07
1507,299,Dead,3.436783e+07
1508,300,Dead,3.437040e+07


### Plot SEIR Model Projection Using Plotly

In [11]:
import plotly.express as px
fig = px.line(state, x='Days', y='Forecast', color='Status', line_group='Status')
fig.show()

## Process Timeseries Data

In [12]:
def get_timeseries_covid_stats(name):
    """return timeseries dataframe"""
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/' \
        f'csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{name}_global.csv'
    df = pd.read_csv(url)
    df.rename(columns={"Country/Region": "Location"}, inplace=True)
    return df

country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [13]:
confirmed_df = get_timeseries_covid_stats("confirmed")
death_df = get_timeseries_covid_stats("deaths")
recovered_df = get_timeseries_covid_stats("recovered")

In [14]:
def get_timeseries_metrics(country="US", count=30):
    df = pd.DataFrame()
    df["Confirmed"] = confirmed_df.groupby("Location").sum().loc[country][-count:].values
    df["Recovered"] = recovered_df.groupby("Location").sum().loc[country][-count:].values
    df["Deaths"] = death_df.groupby("Location").sum().loc[country][-count:].values
    df["Dates"] = confirmed_df.columns[-count:].values
    df = df.melt('Dates')
    df.rename(columns={"variable": "Status", "value": "Forecast"}, inplace=True)
    return df

#### Prepare Timeseries Data for Plotting

In [15]:
result = get_timeseries_metrics()
result

,Dates,Status,Forecast
0,9/19/21,Confirmed,42094648.0
1,9/20/21,Confirmed,42296425.0
2,9/21/21,Confirmed,42417661.0
3,9/22/21,Confirmed,42550081.0
4,9/23/21,Confirmed,42675919.0
...,...,...,...
85,10/14/21,Deaths,721585.0
86,10/15/21,Deaths,723368.0
87,10/16/21,Deaths,723765.0
88,10/17/21,Deaths,724110.0


## Last 30 Days: Timeseries graphs representing growth of COVID in the US

In [16]:
fig = px.bar(result, x="Dates", y="Forecast", color="Status")
fig.show()

In [17]:
fig = px.scatter(result, x="Dates", y="Forecast", color="Status", size="Forecast")
fig.show()

### Timeseries graph showcasing recovered vs. deaths

In [18]:
df = result.loc[result.Status != "Confirmed"]
fig = px.scatter(df, x="Dates", y="Forecast", color="Status", size="Forecast")
fig.show()

In [19]:
df = result.loc[result.Status != "Confirmed"]
fig = px.bar(df, x="Dates", y="Forecast", color="Status", barmode="group")
fig.show()

## Choropleth Map of COVID Spread Across the World

### Helper function for graphing COVID Cases on World Map

In [20]:
# source: folium module website
import folium

def get_world_map(confirmed_df, death_df):
    world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)
    for i in range(0,len(confirmed_df)):
        folium.Circle(
            location=[confirmed_df.iloc[i]['Lat'], confirmed_df.iloc[i]['Long']],
            fill=True,
            radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
            color='red',
            fill_color='indigo',
            tooltip = "<div style='margin: 0; background-color: gray; color: white;'>"+
                        "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['Location'] + "</h4>"
                        "<hr style='margin:10px;color: white;'>"+
                        "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                            "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                            "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                            "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                        "</ul></div>",
            ).add_to(world_map)
    return world_map

In [21]:
world_map = get_world_map(confirmed_df.fillna(0), death_df.fillna(0))
world_map